<a href="https://colab.research.google.com/github/Zig302/ML_testing/blob/main/Prefetch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Imports* and config

In [ ]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# CONFIG
CSV_FILE       = "memory_accesses_pc_va.csv"
HISTORY_LENGTH = 16            # Sliding history window size
TOP_DELTAS     = 128           # K most frequent deltas to predict
BATCH_SIZE     = 512
NUM_EPOCHS     = 50
LEARNING_RATE  = 1e-3
TEST_SIZE      = 0.2           # 20% for validation

Loading, filtering and preparing data

In [ ]:
# LOAD & FILTER
df = pd.read_csv(CSV_FILE)
# Keep only cache misses
df = df[df['cache_hit'] == 0].reset_index(drop=True)  # filter misses

# COMPUTE CACHE-BLOCK & DELTA
# Convert hex string addr -> int, then to 64-byte cache-line index
df['block'] = df['addr'].apply(lambda x: int(x, 16) // 64)
# Predict delta = block_t - block_{t-1}
df['delta'] = df['block'].diff().fillna(0).astype(int)

# BUILD TOP-K DELTA VOCAB & FILTER
top_deltas = df['delta'].value_counts().nlargest(TOP_DELTAS).index
# Keep only rows whose delta is in top-K
df = df[df['delta'].isin(top_deltas)].reset_index(drop=True)
# Map delta -> 0..K-1 label
delta2id = {d:i for i,d in enumerate(top_deltas)}
df['delta_id'] = df['delta'].map(delta2id)

# ENCODE PCs
# Convert hex string ip -> int, then to categorical ID
df['pc_int'] = df['ip'].apply(lambda x: int(x,16))
unique_pcs = df['pc_int'].unique()
pc2id = {pc:i for i,pc in enumerate(unique_pcs)}
df['pc_id'] = df['pc_int'].map(pc2id)

# BUILD SLIDING WINDOWS (history -> next-delta label)
X_pc    = []
X_delta = []
y       = []
for i in range(HISTORY_LENGTH, len(df)):
    # history of PCs and deltas
    X_pc.append(   df['pc_id'].iloc[i-HISTORY_LENGTH:i].values )
    X_delta.append(df['delta_id'].iloc[i-HISTORY_LENGTH:i].values )
    # label is the delta at time i
    y.append( df['delta_id'].iloc[i] )

X_pc    = np.stack(X_pc)        # shape [N_windows, HISTORY_LENGTH]
X_delta = np.stack(X_delta)
y       = np.array(y)

Train/Val split and DataLoader

In [ ]:
# TRAIN/VAL SPLIT
X_pc_tr, X_pc_val, X_delta_tr, X_delta_val, y_tr, y_val = train_test_split(
    X_pc, X_delta, y,
    test_size=TEST_SIZE,
    random_state=42,
    shuffle=True
)

# DATASET & DATALOADER
class PrefetchDeltaDataset(Dataset):
    def __init__(self, pcs, deltas, labels):
        self.pcs    = torch.LongTensor(pcs)
        self.deltas = torch.LongTensor(deltas)
        self.labels = torch.LongTensor(labels)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return (self.pcs[idx], self.deltas[idx]), self.labels[idx]

train_ds = PrefetchDeltaDataset(X_pc_tr, X_delta_tr, y_tr)
val_ds   = PrefetchDeltaDataset(X_pc_val, X_delta_val, y_val)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)

CNN Model

In [ ]:
# 8) MODEL: Deep 1D-CNN on (pc, delta) sequence
class PrefetchCNN(nn.Module):
    def __init__(self, n_pcs, n_deltas,
                 pc_emb=64, delta_emb=16,
                 hidden=128, kernel=3, hist=HISTORY_LENGTH,
                 n_classes=TOP_DELTAS, dropout=0.2):
        super().__init__()
        self.pc_embed    = nn.Embedding(n_pcs,    pc_emb)
        self.delta_embed = nn.Embedding(n_deltas, delta_emb)
        in_ch = pc_emb + delta_emb

        self.conv1 = nn.Conv1d(in_ch, hidden, kernel, padding=kernel//2)
        self.bn1   = nn.BatchNorm1d(hidden)
        self.conv2 = nn.Conv1d(hidden, hidden, kernel, padding=kernel//2)
        self.bn2   = nn.BatchNorm1d(hidden)
        self.conv3 = nn.Conv1d(hidden, hidden, kernel, padding=kernel//2)
        self.bn3   = nn.BatchNorm1d(hidden)

        self.drop = nn.Dropout(dropout)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc   = nn.Linear(hidden, n_classes)

    def forward(self, pc_seq, delta_seq):
        p = self.pc_embed(pc_seq)          # [B, H, pc_emb]
        d = self.delta_embed(delta_seq)    # [B, H, delta_emb]
        x = torch.cat([p, d], dim=2)       # [B, H, in_ch]
        x = x.transpose(1, 2)              # [B, in_ch, H]

        x = F.relu(self.bn1(self.conv1(x)))
        x = self.drop(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.drop(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.drop(x)

        x = self.pool(x).view(x.size(0), -1)
        return self.fc(x)                  # [B, TOP_DELTAS]

Training Loop

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PrefetchCNN(n_pcs=len(unique_pcs), n_deltas=len(top_deltas)).to(device)
opt   = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(1, NUM_EPOCHS+1):
    # Train
    model.train()
    total, correct = 0, 0
    for (pc_b, d_b), y_b in train_loader:
        pc_b, d_b, y_b = pc_b.to(device), d_b.to(device), y_b.to(device)
        opt.zero_grad()
        logits = model(pc_b, d_b)
        loss   = F.cross_entropy(logits, y_b)
        loss.backward(); opt.step()

        preds = logits.argmax(dim=1)
        total += y_b.size(0)
        correct += (preds == y_b).sum().item()
    train_acc = correct / total

    # Validate
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for (pc_b, d_b), y_b in val_loader:
            pc_b, d_b, y_b = pc_b.to(device), d_b.to(device), y_b.to(device)
            preds = model(pc_b, d_b).argmax(dim=1)
            total += y_b.size(0)
            correct += (preds == y_b).sum().item()
    val_acc = correct / total

    print(f"Epoch {epoch:2d} — Train: {train_acc*100:.2f}%  Val: {val_acc*100:.2f}%")

Epoch  1 — Train: 48.95%  Val: 54.78%
Epoch  2 — Train: 55.26%  Val: 60.72%
Epoch  3 — Train: 59.77%  Val: 63.24%
Epoch  4 — Train: 62.44%  Val: 64.68%
Epoch  5 — Train: 64.12%  Val: 65.60%
Epoch  6 — Train: 65.43%  Val: 66.33%
Epoch  7 — Train: 66.47%  Val: 67.02%
Epoch  8 — Train: 67.25%  Val: 67.48%
Epoch  9 — Train: 67.92%  Val: 67.76%
Epoch 10 — Train: 68.59%  Val: 68.07%
Epoch 11 — Train: 69.02%  Val: 68.32%
Epoch 12 — Train: 69.59%  Val: 68.50%
Epoch 13 — Train: 69.95%  Val: 68.66%
Epoch 14 — Train: 70.31%  Val: 68.66%
Epoch 15 — Train: 70.70%  Val: 68.91%
Epoch 16 — Train: 71.06%  Val: 69.14%
Epoch 17 — Train: 71.25%  Val: 69.27%
Epoch 18 — Train: 71.57%  Val: 69.35%
Epoch 19 — Train: 71.88%  Val: 69.43%
Epoch 20 — Train: 72.11%  Val: 69.57%
Epoch 21 — Train: 72.32%  Val: 69.51%
Epoch 22 — Train: 72.56%  Val: 69.63%
Epoch 23 — Train: 72.77%  Val: 69.75%


KeyboardInterrupt: 

Save Weights

In [ ]:
# SAVE MODEL WEIGHTS
MODEL_PATH = "prefetch_cnn_weights.pth"

# Save just the state_dict (Only the learned weights)
torch.save(model.state_dict(), MODEL_PATH)
# torch.save(model, "prefetch_cnn_full.pth") # For the full architecture
print(f"Model weights saved to {MODEL_PATH}")

# DOWNLOAD IN COLAB
from google.colab import files
files.download(MODEL_PATH)
# files.download("prefetch_cnn_full.pth") # Or for the full model

Model weights saved to prefetch_cnn_weights.pth


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>